## Command line version -- (final version that was used to convert to python script)
# Command Line Commands:

cd /Volumes/CDR1TB/dwi_processing_test_cmdline/Manzo_Peter_DTI_test

## Command line version -- final version need to convert to python script
cd /Volumes/CDR1TB/dwi_processing_test_cmdline/Moses_Meghan/Moses_Meghan_DTI

# Convert dti.nii to mrtrix format (creates dti.mif)
mrconvert -fslgrad bvec.bvec bval.bval dti.nii dti.mif -force

# Denoise and view output residual
dwidenoise dti.mif dti_denoise.mif -noise noise.mif -force
mrcalc dti.mif dti_denoise.mif -subtract residual.mif -force
mrview residual.mif

# Extract b0.nii file (creates b0.nii)
dwiextract -bzero dti_denoise.mif b0.mif -force

# Pre-process
dwifslpreproc dti_denoise.mif dti_denoise_preproc.mif -nocleanup -rpe_none -pe_dir AP -eddy_options " --slm=linear --data_is_shelled" -force

mrconvert dti_denoise_preproc.mif dti_denoise_preproc.nii -force

# N4bias correction
dwibiascorrect ants dti_denoise_preproc.mif dti_denoise_preproc_unbiased.mif -bias bias.mif -force
mrview dti_denoise_preproc_unbiased.mif -overlay.load dti.mif
mrconvert dti_denoise_preproc_unbiased.mif dti_denoise_preproc_unbiased.nii -force

# Create brain mask
dwi2mask dti_denoise_preproc_unbiased.mif mask.mif -force
mrconvert mask.mif mask.nii -force

# Fit DTI tensors using dti_denoise_preproc_unbiased.nii, mask.nii, bvec & bval (creates nii.gz files of FA,MD,MO,S0,L1,L2,L3,V1,V2,V3)
dtifit -k dti_denoise_preproc_unbiased.nii -o m -m mask.nii -r bvec.bvec -b bval.bval


## Updated DTI Processing Pipeline using DIPY -- Run in Jupyter Notebook

In [1]:
import os
os.environ['PATH'] += ':/Users/christianraimondo/ants-2.5.0/bin'
nthreads = os.environ.get('MRTRIX_NTHREADS', 'Not set')
print(f'MRTRIX_NTHREADS is {nthreads}')

import tempfile
import subprocess
from tqdm import tqdm
import numpy as np
import nibabel as nib
from dipy.denoise.nlmeans import nlmeans
from dipy.denoise.noise_estimate import estimate_sigma
from dipy.reconst.dti import gradient_table, TensorModel
from dipy.reconst.dti import fractional_anisotropy, mean_diffusivity, color_fa
from dipy.core.gradients import gradient_table
import nibabel as nib
from dipy.reconst.dti import TensorModel, mean_diffusivity, decompose_tensor
import shutil   
from dipy.io import read_bvals_bvecs


def run_command(command, step_name):
    print(f"Running: {step_name}")
    result = subprocess.run(command, shell=True, text=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
    if result.returncode != 0:
        print(f"Command failed with code {result.returncode}: {result.stderr}")
    return result

def denoise_dti(output_dir, dti_path, step_name):
    print(f"Running: {step_name}")
    img = nib.load(dti_path)
    data = img.get_fdata()
    sigma = estimate_sigma(data, N=4)  # N is the number of coils. Adjust as needed
    denoised_data = nlmeans(data, sigma=sigma)
    denoised_img = nib.Nifti1Image(denoised_data, img.affine)
    denoised_dti_path = os.path.join(output_dir, "dti_denoise.nii.gz")
    nib.save(denoised_img, denoised_dti_path)
    return denoised_dti_path

def fit_tensors_with_dipy(output_dir, bvec_path, bval_path, mask_path, step_name):
    print(f"Running: {step_name}")
    denoised_dti_path = os.path.join(output_dir, "dti_denoise.nii.gz")
    dti_img = nib.load(denoised_dti_path)
    data = dti_img.get_fdata()
    affine = dti_img.affine
    
    bvals, bvecs = read_bvals_bvecs(bval_path, bvec_path)
    gtab = gradient_table(bvals, bvecs)
    mask = nib.load(mask_path).get_fdata().astype(bool)
    
    tenmodel = TensorModel(gtab)
    tenfit = tenmodel.fit(data, mask)
    
    # Save the Fractional Anisotropy (FA)
    FA_img = nib.Nifti1Image(tenfit.fa.astype(np.float32), affine)
    nib.save(FA_img, os.path.join(output_dir, "FA.nii.gz"))
    
    # Save primary, secondary, and tertiary eigenvectors (V1, V2, V3)
    for i, v in enumerate([tenfit.evecs[..., 0], tenfit.evecs[..., 1], tenfit.evecs[..., 2]]):
        nib.save(nib.Nifti1Image(v.astype(np.float32), affine), os.path.join(output_dir, f"V{i+1}.nii.gz"))
    
    # Save the Mean Diffusivity (MD), Axial Diffusivity (AD), Radial Diffusivity (RD)
    MD_img = nib.Nifti1Image(tenfit.md.astype(np.float32), affine)
    nib.save(MD_img, os.path.join(output_dir, "MD.nii.gz"))
    
    RD_img = nib.Nifti1Image(tenfit.rd.astype(np.float32), affine)
    nib.save(RD_img, os.path.join(output_dir, "RD.nii.gz"))
    
    AD_img = nib.Nifti1Image(tenfit.ad.astype(np.float32), affine)
    nib.save(AD_img, os.path.join(output_dir, "AD.nii.gz"))
    
    # Save the eigenvalues (L1, L2, L3)
    for i, l in enumerate([tenfit.evals[..., 0], tenfit.evals[..., 1], tenfit.evals[..., 2]]):
        nib.save(nib.Nifti1Image(l.astype(np.float32), affine), os.path.join(output_dir, f"L{i+1}.nii.gz"))


def process_dti(input_dir):
    steps = ["Convert to MRtrix", "Denoising", "Extract b0 Image", "Pre-process", "N4 Bias Correction",
             "Convert to NIFTI for DIPY", "Create Brain Mask", "Tensor Fitting with DIPY"]
    
    output_dir = os.path.join(input_dir, "dti_outputs")
    os.makedirs(output_dir, exist_ok=True)
    
    dti_path = os.path.join(input_dir, "dti.nii.gz")
    bvec_path = os.path.join(input_dir, "bvec.bvec")
    bval_path = os.path.join(input_dir, "bval.bval")
    
    for i, step_name in enumerate(tqdm(steps, desc="Processing DTI Data", unit="step")):
        if step_name == "Convert to MRtrix":
            run_command(f"mrconvert -fslgrad {bvec_path} {bval_path} {dti_path} {output_dir}/nii.mif -force", step_name)
        
        elif step_name == "Denoising":
            denoise_dti(output_dir, dti_path, step_name)
        
        elif step_name == "Extract b0 Image":
            run_command(f"dwiextract -bzero {output_dir}/nii.mif {output_dir}/b0.mif -force", step_name)
        
        elif step_name == "Pre-process":
            eddy_options = "--slm=linear --data_is_shelled"
            run_command(f"dwifslpreproc {output_dir}/nii.mif {output_dir}/dti_denoise_preproc.mif -nocleanup -rpe_none -pe_dir AP -eddy_options \"{eddy_options}\" -force", step_name)
        
        elif step_name == "N4 Bias Correction":
            run_command(f"dwibiascorrect ants {output_dir}/dti_denoise_preproc.mif {output_dir}/dti_denoise_preproc_unbiased.mif -bias {output_dir}/bias.mif -force", step_name)
        
        elif step_name == "Convert to NIFTI for DIPY":
            run_command(f"mrconvert {output_dir}/dti_denoise_preproc_unbiased.mif {output_dir}/dti_denoise_preproc_unbiased.nii -force", step_name)
        
        elif step_name == "Create Brain Mask":
            run_command(f"dwi2mask {output_dir}/dti_denoise_preproc_unbiased.mif {output_dir}/mask.mif -force", step_name)
            run_command(f"mrconvert {output_dir}/mask.mif {output_dir}/mask.nii -force", step_name)
        
        elif step_name == "Tensor Fitting with DIPY":
            fit_tensors_with_dipy(output_dir, bvec_path, bval_path, f"{output_dir}/mask.nii", step_name)



#### Change names below for new patients, make sure they are in the base_path folder
patient_folders = [
    "patient_name",
    "patient_name",
    "patient_name",
    "patient_name",
    "patient_name",
    "patient_name",
    "patient_name",
]

base_path = "/media/jimric/CDR1TB/PD_DWI_PROCESSING/Processing_Laxman/Reprocessing/"

# Ensure the function and script you're using is defined above this loop
for patient in patient_folders:
    patient_path = os.path.join(base_path, patient, f"{patient}_DTI")
    try:
        print(f"Starting Processing of {patient}")
        process_dti(patient_path)
        print(f"Processing of {patient} completed successfully.")
    except Exception as e:
        print(f"Error while processing {patient}: {str(e)}")



MRTRIX_NTHREADS is Not set
Starting Processing of Christburgh_Paris


Processing DTI Data:  12%|█▎        | 1/8 [00:00<00:00, 115.16step/s]


Running: Convert to MRtrix
Command failed with code 1: mrconvert: [WARNING] existing output files will be overwritten
mrconvert: [ERROR] input file "/media/jimric/CDR1TB/PD_DWI_PROCESSING/Processing_Laxman/Reprocessing/Christburgh_Paris/Christburgh_Paris_DTI/bvec.bvec" for option "-fslgrad" not found

Running: Denoising
Error while processing Christburgh_Paris: No such file or no access: '/media/jimric/CDR1TB/PD_DWI_PROCESSING/Processing_Laxman/Reprocessing/Christburgh_Paris/Christburgh_Paris_DTI/dti.nii.gz'
Starting Processing of Christburgh_Paris_2


Processing DTI Data:  12%|█▎        | 1/8 [00:00<00:00, 117.96step/s]


Running: Convert to MRtrix
Command failed with code 1: mrconvert: [WARNING] existing output files will be overwritten
mrconvert: [ERROR] input file "/media/jimric/CDR1TB/PD_DWI_PROCESSING/Processing_Laxman/Reprocessing/Christburgh_Paris_2/Christburgh_Paris_2_DTI/bvec.bvec" for option "-fslgrad" not found

Running: Denoising
Error while processing Christburgh_Paris_2: No such file or no access: '/media/jimric/CDR1TB/PD_DWI_PROCESSING/Processing_Laxman/Reprocessing/Christburgh_Paris_2/Christburgh_Paris_2_DTI/dti.nii.gz'
Starting Processing of Christburgh_Paris_3


...

Running: Convert to MRtrix
Command failed with code 1: mrconvert: [WARNING] existing output files will be overwritten
mrconvert: [ERROR] input file "/media/jimric/CDR1TB/PD_DWI_PROCESSING/Processing_Laxman/Reprocessing/Cohen_David_2/Cohen_David_2_DTI/bvec.bvec" for option "-fslgrad" not found

Running: Denoising
Error while processing Cohen_David_2: No such file or no access: '/media/jimric/CDR1TB/PD_DWI_PROCESSING/Processing_Laxman/Reprocessing/Cohen_David_2/Cohen_David_2_DTI/dti.nii.gz'
Starting Processing of Simmons_Patricia


Processing DTI Data:  12%|█▎        | 1/8 [00:00<00:00, 300.80step/s]


Running: Convert to MRtrix
Command failed with code 1: mrconvert: [WARNING] existing output files will be overwritten
mrconvert: [ERROR] input file "/media/jimric/CDR1TB/PD_DWI_PROCESSING/Processing_Laxman/Reprocessing/Simmons_Patricia/Simmons_Patricia_DTI/bvec.bvec" for option "-fslgrad" not found

Running: Denoising
Error while processing Simmons_Patricia: No such file or no access: '/media/jimric/CDR1TB/PD_DWI_PROCESSING/Processing_Laxman/Reprocessing/Simmons_Patricia/Simmons_Patricia_DTI/dti.nii.gz'
Starting Processing of Simmons_Patricia_2


Processing DTI Data:  12%|█▎        | 1/8 [00:00<00:00, 297.24step/s]

Running: Convert to MRtrix
Command failed with code 1: mrconvert: [WARNING] existing output files will be overwritten
mrconvert: [ERROR] input file "/media/jimric/CDR1TB/PD_DWI_PROCESSING/Processing_Laxman/Reprocessing/Simmons_Patricia_2/Simmons_Patricia_2_DTI/bvec.bvec" for option "-fslgrad" not found

Running: Denoising
Error while processing Simmons_Patricia_2: No such file or no access: '/media/jimric/CDR1TB/PD_DWI_PROCESSING/Processing_Laxman/Reprocessing/Simmons_Patricia_2/Simmons_Patricia_2_DTI/dti.nii.gz'
